In [1]:
import os
import numpy as np
import pandas as pd
import tqdm

In [2]:
save_dir = './processed_data'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [3]:
root_dir = r"source_data"
filepath = os.path.join(root_dir, r'ATLAS.xlsx')
assert os.path.exists(filepath), f'{filepath} not exists!'
df = pd.read_excel(filepath,index_col=0,dtype={'PMID':str})

# PDB: 若有true_PDB，则取true_PDB赋值给PDB，否则取template_PDB赋值给PDB
inds = df[~df['true_PDB'].isna()].index
df.loc[inds,'PDB'] = df.loc[inds,'true_PDB']
inds =  df[df['true_PDB'].isna()&~df['template_PDB'].isna()].index
df.loc[inds,'PDB'] = df.loc[inds,'template_PDB']
## 取值template_PDB的，要留意template_PDB是否和MT序列一致，不一致的话要标注其中突变##
assert sum(df['PDB'].isna())==0

print(f"Original dataset has {len(df)} samples with {len(set(df['PDB'].tolist()))} unique PDBs")

Original dataset has 694 samples with 87 unique PDBs


In [4]:
df['Kd_microM'] = df['Kd_microM'].apply(lambda x: x if x!='n.d.' else np.nan)
df['KD(M)'] = df['Kd_microM'].apply(lambda x: float(str(x).split('>')[-1].split('+/-')[0].strip())*1E-6)


# deleted samples without affinity labels
df_del = df[df['KD(M)'].isna()]
df_del['Reason for Deletion'] = 'without affinity labels'
# save as excel file
df_del.to_excel( os.path.join(save_dir,'ATLAS(samples_deleted).xlsx') )


df = df[~df['KD(M)'].isna()]
print(f"The dataset has {len(df)} samples with known affinity, among which, there are {len(set(df['PDB'].tolist()))} unique PDBs")

The dataset has 637 samples with known affinity, among which, there are 84 unique PDBs


/tmp/ipykernel_51102/3785044251.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_del['Reason for Deletion'] = 'without affinity labels'


In [5]:
df['Source Data Set'] = 'ATLAS'
df['Model'] = np.nan
df['Affinity Release Date'] = np.nan

In [6]:
df.rename( columns={
    'Exp. Method': 'Affinity Method',
    'PMID': 'Affinity PubMed ID',
    'TCRname': 'Receptor Name',
    'Temperature_K': 'Temperature(K)',
    'Resolution': 'Resolution(Å)',
},inplace=True)

df['Affinity PubMed ID'] = df['Affinity PubMed ID'].astype(str)
df['Temperature(K)'] = df['Temperature(K)'].apply(lambda x: x if x!='\\N' else np.nan)
df['Resolution(Å)'] = df['Resolution(Å)'].apply(lambda x: x if x!='\\N' else np.nan)
df['Ligand Name'] = df['PEPseq']+'-'+df['MHCname']

# Add Mutattians

## TCR_mut

In [7]:
TCR_chain_convert = {'A':'D', 'B':'E'}
df['TCR_mut_chain'].fillna(value='', inplace=True)
df['TCR_mut_chain'] = df['TCR_mut_chain'].apply(lambda x: x if x not in ['\\N','N/A'] else '')
df['TCR_mut'] = df['TCR_mut'].apply(lambda x: '' if x=='WT' else x)
for ind in df.index:
    if df.loc[ind,'TCR_mut']=='':
        # assert df.loc[ind,'TCR_mut_chain']=='', f"{df.loc[ind,'PDB']}: TCR_mut_chain must be empty while TCR_mut is empty"
        pass
    else:
        mut_chains = [chain.strip() for chain in df.loc[ind,'TCR_mut_chain'].split('|')]
        muts = [mut.strip() for mut in df.loc[ind,'TCR_mut'].split('|')]
        df.loc[ind,'TCR_mut'] = ', '.join([f'{TCR_chain_convert[mut_chain]}_{mut}' for mut_chain, mut in zip(mut_chains,muts)])
assert len(df[df['TCR_mut'].apply(lambda x: '|' in x if isinstance(x,str) else False)].loc[:,['TCR_mut']]) == 0, "Error: TCR_mut with '|' is note allowed!"

## MHC_mut

In [8]:
MHC_chain_convert = {'A':'A', 'B':'B'}
df['MHC_mut_chain'].fillna(value='', inplace=True)
df['MHC_mut_chain'] = df['MHC_mut_chain'].apply(lambda x: x if x not in ['\\N','N/A'] else '')
df['MHC_mut'] = df['MHC_mut'].apply(lambda x: '' if x=='WT' else x)
for ind in df.index:
    if df.loc[ind,'MHC_mut_chain']=='':
        # assert df.loc[ind,'TCR_mut_chain']=='', f"{df.loc[ind,'PDB']}: TCR_mut_chain must be empty while TCR_mut is empty"
        pass
    else:
        mut_chains = [chain.strip() for chain in df.loc[ind,'MHC_mut_chain'].split('|')]
        muts = [mut.strip() for mut in df.loc[ind,'MHC_mut'].split('|')]
        df.loc[ind,'MHC_mut'] = ', '.join([f'{mut_chain}_{mut}' for mut_chain, mut in zip(mut_chains,muts)])
assert len(df[df['MHC_mut'].apply(lambda x: '|' in x if isinstance(x,str) else False)].loc[:,['MHC_mut']]) == 0, "Error: Mut_mut with '|' is note allowed!"

## PEP_mut

In [9]:
df['PEP_mut'] = df['PEP_mut'].apply( lambda x: ', '.join([x.strip('') for mut in x.split('|') if x!='WT']) )
# # 读取pdb文件，根据序列，找出其中的多肽链
df['PEPseq'] = df['PEPseq'].apply(lambda x: x.strip())

# ATLAS数据集中的多肽链默认是C
df['PEP_mut'] = df['PEP_mut'].apply(lambda x: ', '.join([f'C_{mut.strip()}' for mut in x.split(',') if mut!='']) )

## merge

In [10]:
for ind in df.index:
    muts = [df.loc[ind,'MHC_mut'], df.loc[ind,'PEP_mut'], df.loc[ind,'TCR_mut']] 
    muts = list(filter(lambda x : x !='', muts))
    df.loc[ind,'Mutations'] = ', '.join( muts ).strip(', ')
df[df['Mutations']==''].loc[:,'Mutations'] = np.nan

# Screening

In [11]:
# 精筛：筛除受体链、配体链不是氨基酸链的样本
df.reset_index(inplace=True, drop=True)

from utils import check_chains
assert len(set(df.index.values.tolist())) == len(df)
data_dir = r"processed_data/PDB/ATLAS"
keep_inds = []
for pdb in tqdm.tqdm(set(df.PDB.tolist()),'Iterate over all PDBs'):
    pdb_file = os.path.join( data_dir, f'{pdb.upper()}.pdb' )
    df_tmp = df[df['PDB']==pdb]
    for ind in df_tmp.index:
        ligand_chains = [chain.strip() for chain in df_tmp.loc[ind,'Ligand Chains'].split(',')]
        receptor_chains = [chain.strip() for chain in df_tmp.loc[ind,'Receptor Chains'].split(',')]
        result = check_chains(pdb_file, ligand_chains, receptor_chains)
        if all(result.values()):
            keep_inds.append(ind)
        else:
            print(f"Delete {pdb}, because: {result}")
drop_inds = [ind for ind in df.index.tolist() if ind not in keep_inds]
df_drop = df.loc[drop_inds]
pdbs_drop = set(df_drop['PDB'].values.tolist())
print(f'Number of dropped PDBs: {len(pdbs_drop)}, which is:\n{pdbs_drop}')
df = df.loc[keep_inds]
print('After the filtering, we get df:')
display(df)

Iterate over all PDBs: 100%|██████████| 84/84 [00:31<00:00,  2.64it/s]

Number of dropped PDBs: 0, which is:
set()
After the filtering, we get df:


,Receptor Name,MHCname,MHCname_PDB,MHC_mut,MHC_mut_chain,TCR_mut,Kd_microM,Kon_per_M_per_s,Koff_per_s,Kd_wt/Kd_mut,...,Immobilized Ligand,Coupling Method,Analyte,PDB,KD(M),Source Data Set,Model,Affinity Release Date,Ligand Name,Mutations
477,MEL5,HLA-A*02:01,HLA-A*02:01,,,,6.4+/-0.2,NaN,NaN,NaN,...,EAAGIGILTV-HLA-A*02:01,indirect,MEL5 WT,4QOK,0.000006,ATLAS,NaN,NaN,EAAGIGILTV-HLA-A*02:01,
594,2C,H-2Kb,H-2Kb,,,,84.1+/-12.0,2200,18.5,NaN,...,2C WT,direct,EQYKFYSV-H-2KB,1MWA,0.000084,ATLAS,NaN,NaN,EQYKFYSV-H-2Kb,
588,BM3.3,H-2Kb,H-2Kb,,,,2.6,NaN,NaN,NaN,...,INFDFNTI-H-2Kb,indirect,BM3.3 WT,1FO0,0.000003,ATLAS,NaN,NaN,INFDFNTI-H-2Kb,
591,BM3.3,H-2Kb,H-2Kb,,,,3.6,280000,1.1,NaN,...,INFDFNTI-H-2Kb,indirect,BM3.3 WT,1FO0,0.000004,ATLAS,NaN,NaN,INFDFNTI-H-2Kb,
587,C7,HLA-A*02:01,HLA-A*02:02,,,,5.1,NaN,NaN,NaN,...,NLVPMVATV-HLA-A*02:01,indirect,C7 WT,5D2L,0.000005,ATLAS,NaN,NaN,NLVPMVATV-HLA-A*02:01,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,2B4,I-Ek,I-Ek,,,,240,2070,0.5,0.02,...,ANERADLIAYLRQATK-IEk,indirect,2B4 WT,3QIB,0.000240,ATLAS,NaN,NaN,ADLIAYLRQATKG-I-Ek,C_K8R
633,2B4,I-Ek,I-Ek,,,,33.8,3950,0.134,0.16,...,ANERADLIAYLKQASK-IEk,indirect,2B4 WT,3QIB,0.000034,ATLAS,NaN,NaN,ADLIAYLKQASKG-I-Ek,C_T11S
634,2B4,I-Ek,I-Ek,,,,43.4,4030,680,0.13,...,ANERADLIAYLKQANK-IEk,indirect,2B4 WT,3QIB,0.000043,ATLAS,NaN,NaN,ADLIAYLKQANKG-I-Ek,C_T11N
635,2B4,I-Ek,I-Ek,,,,42.9,4200,0.18,0.13,...,ANERADLIAFLKQASK-IEk,indirect,2B4 WT,3QIB,0.000043,ATLAS,NaN,NaN,ADLIAFLKQASKG-I-Ek,"C_Y6F | T11S, C_Y6F | T11S"


# pdb info

In [12]:
from utils import get_pdb_info
pdbs = set(df['PDB'].values)

for pdb in tqdm.tqdm(pdbs, desc='Adding PDB informations'):
    results = get_pdb_info(pdb)
    if results['Resolution']!=None:
        df.loc[df[df.PDB==pdb].index,'Resolution(Å)'] = results['Resolution'].replace('Å','').strip()
    if results['Method'] != None:
        df.loc[df[df.PDB==pdb].index,'Structure Method'] = results['Method']
    if results['Release Date'] != None:
        df.loc[df[df.PDB==pdb].index,'PDB Release Date'] = results['Release Date']   
    # 这是PDB的PubMed ID, 不是亲和力的PubMed ID
    if results['PubMed ID'] != None:
        df.loc[df[df.PDB==pdb].index,'PDB PubMed ID'] = results['PubMed ID']

Adding PDB informations: 100%|██████████| 84/84 [02:14<00:00,  1.60s/it]


# affinity info

In [13]:
# 根据Affinity PubMed ID查询Affinity Release Date
from utils import get_pubdate
for pubmed_id in tqdm.tqdm(set(df['Affinity PubMed ID'].values), desc='Adding affinity informations'):
    try:
        pubdate = get_pubdate(str(pubmed_id), dst_datetime_format='%Y-%m-%d')
        df.loc[df[df['Affinity PubMed ID']==pubmed_id].index, 'Affinity Release Date'] = pubdate
    except:
        pass

Adding affinity informations: 100%|██████████| 67/67 [00:59<00:00,  1.12it/s]


# save

In [14]:
df.reset_index(drop=True, inplace=True)
df = df.loc[:,[
    'PDB', 'Source Data Set', 'Model', 'Mutations', 'Ligand Chains', 'Receptor Chains', 'Ligand Name', 'Receptor Name', 'KD(M)', 
    'Affinity Method', 'Structure Method', 'Temperature(K)', 'Resolution(Å)', 
    'PDB PubMed ID', 'PDB Release Date', 'Affinity PubMed ID', 'Affinity Release Date',
    
]]
display(df)
save_dir = './processed_data'
df.to_excel( os.path.join(save_dir,'ATLAS.xlsx') )

,PDB,Source Data Set,Model,Mutations,Ligand Chains,Receptor Chains,Ligand Name,Receptor Name,KD(M),Affinity Method,Structure Method,Temperature(K),Resolution(Å),PDB PubMed ID,PDB Release Date,Affinity PubMed ID,Affinity Release Date
0,4QOK,ATLAS,NaN,,"C,A","D,E",EAAGIGILTV-HLA-A*02:01,MEL5,0.000006,Biacore T100,X-RAY DIFFRACTION,298.00,3.00,25471691,2014-12-17,25471691,NaN
1,1MWA,ATLAS,NaN,,"C,A","D,E",EQYKFYSV-H-2Kb,2C,0.000084,Biacore 2000,X-RAY DIFFRACTION,298.15,2.40,11994422,2002-11-27,10755612,NaN
2,1FO0,ATLAS,NaN,,"C,A","D,E",INFDFNTI-H-2Kb,BM3.3,0.000003,Biacore 2000,X-RAY DIFFRACTION,298.15,2.50,11017099,2000-10-02,11017099,NaN
3,1FO0,ATLAS,NaN,,"C,A","D,E",INFDFNTI-H-2Kb,BM3.3,0.000004,Biacore,X-RAY DIFFRACTION,298.15,2.50,11017099,2000-10-02,17363906,NaN
4,5D2L,ATLAS,NaN,,"C,A","D,E",NLVPMVATV-HLA-A*02:01,C7,0.000005,Biacore T100,X-RAY DIFFRACTION,298.15,3.51,26429912,2015-10-07,26429912,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,3QIB,ATLAS,NaN,C_K8R,"C,A,B","D,E",ADLIAYLRQATKG-I-Ek,2B4,0.000240,Biacore,X-RAY DIFFRACTION,298.15,2.70,21490152,2011-04-27,12152083,NaN
633,3QIB,ATLAS,NaN,C_T11S,"C,A,B","D,E",ADLIAYLKQASKG-I-Ek,2B4,0.000034,Biacore,X-RAY DIFFRACTION,298.15,2.70,21490152,2011-04-27,12152083,NaN
634,3QIB,ATLAS,NaN,C_T11N,"C,A,B","D,E",ADLIAYLKQANKG-I-Ek,2B4,0.000043,Biacore,X-RAY DIFFRACTION,298.15,2.70,21490152,2011-04-27,12152083,NaN
635,3QIB,ATLAS,NaN,"C_Y6F | T11S, C_Y6F | T11S","C,A,B","D,E",ADLIAFLKQASKG-I-Ek,2B4,0.000043,Biacore,X-RAY DIFFRACTION,298.15,2.70,21490152,2011-04-27,12152083,NaN
